In [39]:
import os
import pandas as pd
import soccerdata as sd
from tqdm import tqdm

In [76]:
# Primero Sofascore
folder1 = "Sofascore"
ligas = ["Premier","Ligue_1","Bundesliga"]
años = ["19-20", "20-21", "21-22", "22-23", "23-24"]
torneos = []

for liga in ligas:
    for año in años:
        match_file = f"stats_{año}.csv"
        try:
            path_matches = os.path.join(folder1, liga, match_file)
            matches = pd.read_csv(path_matches)
            matches["Date1"] = f"20{año[:2]}/01/01"
            matches["Date2"] = f"20{año[3:5]}/01/01"
        except FileNotFoundError:
            continue
        torneos.append(matches)
df_sofascore = pd.concat(torneos).add_prefix("Sofascore_")
df_sofascore['Sofascore_Date1'] = pd.to_datetime(df_sofascore['Sofascore_Date1'])
df_sofascore['Sofascore_Date1'] = df_sofascore["Sofascore_Date1"].dt.tz_localize('UTC')
df_sofascore['Sofascore_Date2'] = pd.to_datetime(df_sofascore['Sofascore_Date2'])
df_sofascore['Sofascore_Date2'] = df_sofascore["Sofascore_Date2"].dt.tz_localize('UTC')

In [84]:
df_sofascore["Sofascore_Date1"]=="2019"

0       True
1       True
2       True
3       True
4       True
       ...  
301    False
302    False
303    False
304    False
305    False
Name: Sofascore_Date1, Length: 3157, dtype: bool

In [78]:
df_sofascore[Sofascore_Date1]

NameError: name 'Sofascore_Date1' is not defined

In [15]:
sd.MatchHistory.available_leagues()

['ARG-PrimeraDivision',
 'BRA-Brasileirao',
 'CHI-PrimeraDivision',
 'ENG-Premier League',
 'ESP-La Liga',
 'FRA-Ligue 1',
 'GER-Bundesliga',
 'ITA-Serie A',
 'NED-Eredivisie',
 'RUS-PremierLeague',
 'SCO-Premiership',
 'TUR-SuperLig',
 'USA-MLS']

In [16]:
folder2 = "FootballData"
ligas = [
    'ENG-Premier League',
    'ESP-La Liga',
    'FRA-Ligue 1',
    'GER-Bundesliga',
    'ITA-Serie A']
torneos = []
for liga in ligas:
    match_file = f"FootballData_{liga}.csv"
    try:
        path_matches = os.path.join(folder2, match_file)
        matches = pd.read_csv(path_matches)
    except FileNotFoundError:
        continue
    torneos.append(matches)

df_FootballData = pd.concat(torneos)

df_FootballData = df_FootballData.drop(
    ['Unnamed: 0.1'], axis=1).add_prefix("FootballData_")

# Convertir la columna 'Fecha' a tipo datetime
df_FootballData['FootballData_Date'] = pd.to_datetime(df_FootballData['FootballData_Date'], format='%d/%m/%Y')

# Convertir la zona horaria a UTC
df_FootballData['FootballData_Date'] = df_FootballData['FootballData_Date'].dt.tz_localize('UTC')

In [17]:
folder3 = "ESPN"
ligas = [
    'ENG-Premier League',
    'ESP-La Liga',
    'FRA-Ligue 1',
    'GER-Bundesliga',
    'ITA-Serie A']
torneos = []
for liga in ligas:
    match_file = f"ESPN_{liga}.csv"
    try:
        path_matches = os.path.join(folder3, match_file)
        matches = pd.read_csv(path_matches)
        matches = matches.drop("roster",axis=1)
    except FileNotFoundError:
        continue
    torneos.append(matches)
df_ESPN = pd.concat(torneos)
df_ESPN = df_ESPN.drop(["season","Unnamed: 0"], axis = 1)
df_ESPN = df_ESPN.drop_duplicates()


In [18]:
games = df_ESPN["game"].unique()
resultados = []
for i in  tqdm(range(len(games))):
    aux = df_ESPN[df_ESPN["game"]==games[i]]
    away = aux[aux["is_home"] == False].add_prefix("away_")
    home = aux[aux["is_home"] == True].add_prefix("home_")
    resultados.append(pd.DataFrame(pd.concat([home.iloc[0],away.iloc[0]])).T)

  0%|          | 0/35761 [00:00<?, ?it/s]

100%|██████████| 35761/35761 [02:09<00:00, 277.01it/s]


In [19]:
df_ESPN = pd.concat(resultados).add_prefix("ESPN_")

In [20]:
df_ESPN[['Fecha', 'Estadio']] = df_ESPN['ESPN_home_game'].str.split(' ', n=1, expand=True)
# Convertir la columna 'Fecha' a tipo datetime
df_ESPN['Fecha'] = pd.to_datetime(df_ESPN['Fecha'])

# Convertir la zona horaria a UTC
df_ESPN['Fecha'] = df_ESPN['Fecha'].dt.tz_localize('UTC')


In [21]:
df_FootballData = df_FootballData.rename(columns={'FootballData_HomeTeam': 'HomeTeam',"FootballData_FTHG":"HomeGoals",
                                                  'FootballData_AwayTeam': 'AwayTeam', "FootballData_FTAG":"AwayGoals",
                                                  'FootballData_Date':"Date"})

df_sofascore = df_sofascore.rename(columns={'Sofascore_team_home': 'HomeTeam',"Sofascore_home_score":"HomeGoals",
                                            'Sofascore_team_away': 'AwayTeam', "Sofascore_away_score":"AwayGoals"})

df_ESPN = df_ESPN.rename(columns={'ESPN_home_team': 'HomeTeam','ESPN_away_team': 'AwayTeam',
                                  "Fecha":"Date" })

### Superkey

In [38]:
key_FootballData = len(df_FootballData[["HomeTeam","HomeGoals","Date"]].drop_duplicates()) == len(df_FootballData)
key_ESPN = len(df_FootballData[["HomeTeam","HomeGoals","Date"]].drop_duplicates()) == len(df_FootballData)
print(F"FootballData key : {key_FootballData}")
print(F"ESPN key : {key_ESPN}")

FootballData key : True
ESPN key : True


In [22]:
prueba = df_sofascore.merge(df_FootballData, on = ["HomeTeam","AwayTeam","HomeGoals","AwayGoals"], how = "outer")
prueba.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57015 entries, 0 to 57014
Columns: 226 entries, Sofascore_number_of_match to FootballData_SYA
dtypes: datetime64[ns, UTC](1), float64(218), object(7)
memory usage: 98.3+ MB


In [23]:
prueba = df_ESPN.merge(df_FootballData, on = ["HomeTeam","AwayTeam","Date"], how = "outer")
prueba.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79211 entries, 0 to 79210
Columns: 238 entries, ESPN_home_league to FootballData_SYA
dtypes: datetime64[ns, UTC](1), float64(153), object(84)
memory usage: 143.8+ MB


In [24]:
df_ESPN

,ESPN_home_league,ESPN_home_game,HomeTeam,ESPN_home_is_home,ESPN_home_venue,ESPN_home_attendance,ESPN_home_capacity,ESPN_home_fouls_committed,ESPN_home_yellow_cards,ESPN_home_red_cards,...,ESPN_away_tackle_pct,ESPN_away_interceptions,ESPN_away_effective_clearance,ESPN_away_total_clearance,ESPN_away_goal_difference,ESPN_away_total_goals,ESPN_away_goal_assists,ESPN_away_goals_conceded,Date,Estadio
0,ENG-Premier League,2001-08-18 Charlton Athletic-Everton,Charlton Athletic,True,The Valley,20451.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,2001-08-18 00:00:00+00:00,Charlton Athletic-Everton
0,ENG-Premier League,2001-08-18 Derby County-Blackburn Rovers,Derby County,True,Pride Park Stadium,28236.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,2001-08-18 00:00:00+00:00,Derby County-Blackburn Rovers
0,ENG-Premier League,2001-08-18 Leeds United-Southampton,Leeds United,True,Elland Road,39715.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,2001-08-18 00:00:00+00:00,Leeds United-Southampton
0,ENG-Premier League,2001-08-18 Leicester City-Bolton Wanderers,Leicester City,True,Filbert Street,19987.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,2001-08-18 00:00:00+00:00,Leicester City-Bolton Wanderers
0,ENG-Premier League,2001-08-18 Liverpool-West Ham United,Liverpool,True,Anfield,43935.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,2001-08-18 00:00:00+00:00,Liverpool-West Ham United
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,ITA-Serie A,2023-06-04 Atalanta-Monza,Atalanta,True,Gewiss Stadium,19389.0,0.0,13.0,1.0,0.0,...,0.8,8.0,13.0,13.0,NaN,NaN,NaN,NaN,2023-06-04 00:00:00+00:00,Atalanta-Monza
0,ITA-Serie A,2023-06-04 Lecce-Bologna,Lecce,True,Stadio Ettore Giardiniero - Via del Mare,25816.0,0.0,11.0,1.0,0.0,...,0.5,4.0,13.0,13.0,NaN,NaN,NaN,NaN,2023-06-04 00:00:00+00:00,Lecce-Bologna
0,ITA-Serie A,2023-06-04 Napoli-Sampdoria,Napoli,True,Diego Armando Maradona,51000.0,0.0,7.0,0.0,0.0,...,0.8,10.0,22.0,22.0,NaN,NaN,NaN,NaN,2023-06-04 00:00:00+00:00,Napoli-Sampdoria
0,ITA-Serie A,2023-06-04 Udinese-Juventus,Udinese,True,Bluenergy Stadium,23266.0,0.0,9.0,2.0,0.0,...,0.7,7.0,14.0,14.0,NaN,NaN,NaN,NaN,2023-06-04 00:00:00+00:00,Udinese-Juventus
